In [ ]:
import pygame   # 직관적인 ?로 심플한 게임을 만들 수 있는 모듈
import random

pygame.init()

# 전역 변수/상수는 프로그램 전체 구간에서 참조할 수 있다.
SCREEN_WIDTH = 800      # 팝업창의 크기를 결정, 대문자로 적은 이유는 상수 역할을 하기 때문 / 따라서 상수는 모두 대문자로 정의한다.
SCREEN_HEIGHT = 600

# color 정의    / PC에서 색깔은 (R, G, B)순으로 정의된다. 255가 최대, 0이 최소. 따라서 검정은 (0, 0, 0) 이런 식으로 정의됨.
WHITE = (255, 255, 255)     # 이 또한 전역변수 / tuple 속성
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)         # 색깔은 Red, Green, Blue 순 3원색 요소로 정해짐.
RED = (255, 0, 0)

BLOCK_SIZE = 20     # 사용자 지정 1 pixel 값
INITIAL_SPEED = 15  # snake의 초기 속도

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))   # pygame과 display는 모듈임. 튜플 형태로 값을 전달하기 위해 괄호를 2개 사용.
pygame.display.set_caption('Snake Game')
clock = pygame.time.Clock()     # snake 속도제어

class GameOverException(Exception):
    pass

class GameObject:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def draw(self, color):  # 블럭 정보
        pygame.draw.rect(screen, color, [self.x, self.y, BLOCK_SIZE, BLOCK_SIZE])

class Food(GameObject):
    def __init__(self):
        x = random.randint(0, (SCREEN_WIDTH - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE      # 먹이가 선 넘어 생기는걸 방지 / 곱하고 나누는 이유는 random 하게 적은 개수만 뽑기 위해서.
        y = random.randint(0, (SCREEN_HEIGHT - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
        super().__init__(x, y)

class Snake(GameObject):
    def __init__(self):
        super().__init__(SCREEN_HEIGHT // 2, SCREEN_HEIGHT // 2)    # 가운데서 출발
        self.direction = "RIGHT"
        self.body =[]
        self.length = 1

    def move(self):
        if self.direction == 'UP':
            self.y -= BLOCK_SIZE
        if self.direction == 'DOWN':
            self.y += BLOCK_SIZE
        if self.direction == 'LEFT':
            self.x -= BLOCK_SIZE
        if self.direction == 'RIGHT':
            self.x += BLOCK_SIZE

        self.body.insert(0, [self.x, self.y])
        if len(self.body) > self.length:
            self.body.pop()

    def change_dir(self, direction):
        if direction == 'UP' and not self.direction == 'DOWN':      # direction은 새로 지정된 방향, self.direction은 기존의 방향
            self.direction = 'UP'
        if direction == 'DOWN' and not self.direction == 'UP':
            self.direction = 'DOWN'
        if direction == 'LEFT' and not self.direction == 'RIGHT':
            self.direction = 'LEFT'
        if direction == 'RIGHT' and not self.direction == 'LEFT':
            self.direction = 'RIGHT'

    def grow(self):
        self.length += 1

    def draw(self, color):
        for block in self.body:
            pygame.draw.rect(screen, color, [block[0], block[1], BLOCK_SIZE, BLOCK_SIZE])

def game_loop():    # 무한루프가 돌아가며 창을 유지시켜주는 함수.
    global INITIAL_SPEED    # 전역 변수를 다른 범위 내에서 수정할 수 있게 해주는 선언
    game_over = False
    snake = Snake()
    food = Food()

    while not game_over:
        for event in pygame.event.get():    # 프로그램에 저장되어 있는 여러 기능이 있음. 예를 들어 버튼 클릭이라던가 마우스이동이라던가..
            if event.type == pygame.QUIT:   # 닫기 버튼 누른거임.
                pygame.quit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    snake.change_dir('UP')
                elif event.key == pygame.K_DOWN:
                    snake.change_dir('DOWN')
                elif event.key == pygame.K_LEFT:
                    snake.change_dir('LEFT')
                elif event.key == pygame.K_RIGHT:
                    snake.change_dir('RIGHT')

        if snake.x > SCREEN_WIDTH or snake.x < 0 or snake.y > SCREEN_WIDTH or snake.y < 0:
            raise GameOverException("Snake hit the boundary")

        if snake.x == food.x and snake.y == food.y:
            food = Food()
            INITIAL_SPEED += 2
            snake.grow()

        snake.move()

        screen.fill(BLACK)
        snake.draw(RED)
        food.draw(WHITE)
        # go = GameObject(10, 20)     # 10, 20은 블럭의 시작 좌표인듯
        # go.draw(RED)    # 블럭 색깔 지정
        pygame.display.update()

        clock.tick(INITIAL_SPEED)


if __name__ == "__main__":
    print("starting game...")
    game_loop()
